<h1>Relatório: Aprendizado de máquina e reconhecimento de padrões</h1>
<h3>Aluno: André de Sousa Araújo</h3>
<p>Objetivo principal é estimar a ocorrências de precipações extremas.</p>
<p>Espaço: Região HOUSTON - Texas</p>
<p><b>Base escolhida:</b>
This public dataset was created by the National Oceanic and Atmospheric Administration (NOAA) and includes global data obtained from the USAF Climatology Center.  This dataset covers GSOD data between 1929 and present, collected from over 9000 stations.
</p>
<p>Dataset Source: NOAA</p>

<p>Category: Weather</p>

<p>Use: This dataset is publicly available for anyone to use under the following terms provided by the Dataset Source — http://www.data.gov/privacy-policy#data_policy — and is provided "AS IS" without any warranty, express or implied, from Google. Google disclaims all liability for any damages, direct or indirect, resulting from the use of the dataset.</p>

<p>Update Frequency: daily</p>

<h2 style="color:blue;">Importando o dataset e explorando os dados</h2>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
import datalab.bigquery as bq
import seaborn as sns
import pandas as pd
import numpy as np
import shutil

<h2 style="color:blue;">Criando o dataset para treinamento e validação</h2>

<p>Vamos selecionar para treinamento e validação os meses de agosto de 1985 até 2015 desta região.</p>

In [3]:
def create_query(year):
  base_query = """
       SELECT    d.stn
                 ,ws.lat
                 ,ws.lon
                 ,d.da 
                 ,d.mo
                 ,d.year 
                 ,d.temp
                 ,d.dewp
                 ,d.slp
                 ,d.stp
                 ,d.visib
                 ,d.wdsp
                 ,d.mxpsd
                 ,d.gust
                 ,d.max
                 ,d.min
                 ,d.prcp
                 ,d.flag_prcp
                 ,d.sndp
                 ,d.fog
                 ,d.rain_drizzle
                 ,d.snow_ice_pellets
                 ,d.hail
                 ,d.thunder
                 ,d.tornado_funnel_cloud
          FROM 
            [bigquery-public-data:noaa_gsod.gsod{0}] d
            JOIN [bigquery-public-data:noaa_gsod.stations] ws
              ON d.stn = ws.usaf
              and d.wban = ws.wban
          where ws.state is not null
              and ws.country = 'US'
              and INTEGER(d.mo) in (6,7,8)
          order by d.stn, d.da
    """
  query = base_query.format(year)
  return query

df = pd.DataFrame([])

for y in range(2015,2016):
  r = bq.Query(create_query(y)).to_dataframe()
  df = df.append(r)

<p>Salvando os dados</p>

In [4]:
def to_csv(df, filename, target):
  outdf = df.copy(deep=False)
  outdf.loc[:, 'key'] = np.arange(0, len(outdf)) # rownumber as key
  # reorder columns so that target is first column
  cols = outdf.columns.tolist()
  cols.remove(target)
  cols.insert(0, target)
  #print (cols)  # new order of columns
  outdf = outdf[cols]
  outdf.to_csv(filename, header=True, index_label=False, index=False)

to_csv(df, '../data/prcp-data-2015.csv','d_prcp')

<h3 style="color:red;">Verificando a distribuição do conjunto de treinamento e validação</h3>

In [ ]:
import matplotlib.pyplot as plt  
%matplotlib inline
plt.rcParams['figure.figsize'] = [14, 8]  
prcp30_limpo = df[df['d_prcp']< 99.99]
prcp30_limpo.d_prcp.hist()  
plt.title(u'Distribuição da precipitação - AGO 1985 a 2015 - região Houston')  
plt.xlabel(u'Precipitação (polegadas)')  
plt.show()  

<h3 style="color:red;">Limiar para classificar a precipitação como extrema do conjunto de treinamento e validação</h3>

In [ ]:
import numpy as np
p = np.percentile(prcp30_limpo['d_prcp'], 99) # return 99%th percentile
print p

<p>Considerando os últimos 30 anos o limiar é 1.65 polegadas. Comparando com 2016 é limiar mais baixo, 2016 teve um agosto bem mais chuvoso e com mais eventos extremos que a média dos últimos anos</p>

<h3 style="color:red;">Medindo a quantidade de chuvas extremas no período</h3>

In [ ]:
extreme = prcp30_limpo[prcp30_limpo['d_prcp']> 1.3] 
r = extreme.groupby(['d_year'])['d_prcp'].count()
r.plot.bar()
plt.title(u'Preciptações extremas - AGO 1985 a 2015 - região Houston - limiar (99%)')  
plt.ylabel(u'Qtde')  
plt.xlabel(u'Ano')  
plt.show()

<h2 style="color:blue;">Feature engineering</h2>

<p>O objetivo é realizar a previsão destes eventos usando as variáveis de 3 dias atrás. No estudo de Dolif Neto e Nobre (2012) foram extraídas variavéis também de 3 dias atrás dos fenômenos precipitações extremas. Assim vamos derivar as variáveis de 3 dias atrás antes do fenômeno. Inicialente <b>vamos derivar somente a precipitação</b>, testar e depois fazer para todas as varíaveis</p>

In [ ]:
prcp30_limpo = prcp30_limpo.apply(pd.to_numeric, errors='coerce')  

tmp = prcp30_limpo[['d_stn','d_da','d_mo','d_year','d_prcp']]
tmp = tmp.sort_values(['d_year','d_stn','d_da'])
tmp = tmp.reset_index(drop=True)

In [ ]:
N = 1 #1 dia antes
feature = 'd_prcp'

nth_prior =[]

# Devirando a variavel para uma lista representando a enesima valor da variavel, se for o primeiro dia não terá dia anterior.
for y in range(2015,2016):
  df_year = tmp[tmp['d_year']== y]
  df_year = df_year.reset_index(drop=True)
  rows = df_year.shape[0]
  prior= [None]*N + [df_year[feature][i-N] for i in range(N, rows)]
  nth_prior = nth_prior + prior
  
col_name = "{}_{}".format(feature, N)  
tmp[col_name] = nth_prior  
tmp[:10]   

<p>Veja que para a estação ID 722430 no dia 4/8/1985, a precipitação do dia anterior (d_prcp_1) é exatamente o valor medido no dia 03/08/1985. Agora vamos fazer isto para todas as variaveis.</p>

In [ ]:
def derivar_valor_anterior(df, dias, variavel):
  N = dias
  feature = variavel
  tmp = df

  nth_prior =[]

  # Devirando a variavel para uma lista representando a enesima valor da variavel, se for o primeiro dia não terá dia anterior.
  for y in range(2015,2016):
    df_year = tmp[tmp['d_year']== y]
    df_year = df_year.reset_index(drop=True)
    rows = df_year.shape[0]
    prior= [None]*N + [df_year[feature][i-N] for i in range(N, rows)]
    nth_prior = nth_prior + prior

  col_name = "{}_{}".format(feature, N)  
  tmp[col_name] = nth_prior


df_dev = df.apply(pd.to_numeric, errors='coerce')  
df_dev = df_dev.sort_values(['d_year','d_stn','d_da'])
df_dev = df_dev.reset_index(drop=True)
  
for var in list(df_dev):
  if var not in ['d_stn','d_da','d_mo','d_year']:
    for N in range(1, 4):
      derivar_valor_anterior(df_dev,N,var)

df_dev.columns 

<p>Pronto! Agora temos variaveis respectivas de 1, 2, 3 dias atrás. Exemplo: d_prcp_1, d_prcp_2, d_prcp_3 referentes a precipitação 1, 2 e 3 dias atrás respectivamente.</p>

<h2 style="color:blue;">Limpeza dos dados</h2>

<p>Conforme dito anteriormente, existem muitos dados ausentes para algumas variáveis. Podemos excluir as instâncias, mas precisamos ver se perderíamos muitos dados</p>

In [ ]:
df_dev = df_dev.apply(pd.to_numeric, errors='coerce')  
df_dev.info()

In [ ]:
df_dev[:10]

In [ ]:
df_temp = df_dev[(df_dev < 9999.9000).all(axis=1)]
print ('Tamanho antes da limpeza: %s' %str(len(df_dev))) 
print ('Tamanho depois da limpeza: %s' %str(len(df_temp))) 

<p>A conclusão é que todo registro tem algum dado ausente. Assim vamos verificar se tem alguma variável que apresenta maiores problemas.<p/>

In [ ]:
def contabilizar_dados_ausentes(df):
  dn = {}
  
  for v in df.columns:
    df_grp = df[df[v] == 9999.9]
    dn[v] = len(df_grp)

  for v in ['d_gust','d_gust_1', 'd_gust_2', 'd_gust_3']:
    if v in df.columns:
      df_grp = df[df[v] == 999.9]
      dn[v] = dn[v] + len(df_grp)

  for v in ['d_prcp','d_prcp_1', 'd_prcp_2', 'd_prcp_3']:
    if v in df.columns:
      df_grp = df[df[v] == 99.9]
      dn[v] = dn[v] + len(df_grp) 
    
  pdn = pd.DataFrame(dn.items(), columns=['freatures', 'missing'])
  pdn = pdn[pdn > 0]
  pdn = pdn.sort_values(['missing'])
  
  return pdn

pdn = contabilizar_dados_ausentes(df_dev)
pdn.plot.bar(x='freatures', y='missing')
plt.show()

<p> As varíveis velocidade máxima do vento do dia (d_mxpsd), velocidade do fenômeno de aumento repentino do vento (d_gust), média da velocidade do vento (d_wdsp), temperatura do ponto de orvaloho (d_dewp) e pressão atmosferica (d_stp) apresentam valores nulos significativos. Sendo a variável d_gust a mais problemática.</p>

<p>A decisão será remover velocidade do fenômeno de aumento repentino do vento (d_gust) e  pressão atmosferica (d_stp). E interpolar as demais utilizando algum método.</p>

In [ ]:
df_limpo = df_dev.copy(deep=True)

for column in df_limpo.columns:
    if column in ['d_gust','d_gust_1','d_gust_2','d_gust_3']:
        df_limpo = df_limpo.drop(column, axis = 1)
        
for column in df_limpo.columns:
    if column in ['d_stp','d_stp_1','d_stp_2','d_stp_3']:
        df_limpo = df_limpo.drop(column, axis = 1)
        
for column in df_limpo.columns:
    if column in ['d_flag_prcp','d_flag_prcp_1','d_flag_prcp_2','d_flag_prcp_3']:
        df_limpo = df_limpo.drop(column, axis = 1)
        
for column in df_limpo.columns:
    if column in ['d_sndp','d_sndp_1','d_sndp_2','d_sndp_3']:
        df_limpo = df_limpo.drop(column, axis = 1)

df_limpo.columns

<p>Agora temos outras varíaveis para tratar, que ainda estão afetando muitas instâncias</p>

In [ ]:
p = contabilizar_dados_ausentes(df_limpo)
p.plot.bar(x='freatures', y='missing')
plt.show()

<p>Primeiramente vamos marcar os dados ausentes.</p>

In [ ]:
def marcar_dados_ausentes(df):
  for v in df.columns:
    df[v] = df[v].replace([9999.9],"NaN")
  for v in df.columns:
    df[v] = df[v].replace([999.9],"NaN")

marcar_dados_ausentes(df_limpo)

for v in df_limpo.columns:
  if v in ['d_prcp','d_prcp_1', 'd_prcp_2', 'd_prcp_3']:
    df_limpo[v] = df_limpo[v].replace([99.9],"NaN")

df_limpo = df_limpo.apply(pd.to_numeric, errors='coerce')  
df_limpo.info()

<p>Interpolá-los</p>

<h2 style="color:blue;">Interpolandos dados para outras variáveis: dados ausentes</h2>

In [ ]:
df_intptd = df_limpo.copy(deep=True)
for v in df_intptd:
  if v not in ['d_stn','d_da','d_mo','d_year']:
    df_intptd[v] = df_intptd[v].astype(float)

df_intptd = df_intptd.interpolate(method='linear', axis=0).ffill().bfill()
df_intptd.info()

In [ ]:
df_intptd[:10]

<p>Adicionando um flag para indicar se a chuva é extrema ou não</p>

<h2 style="color:blue;">Adicionando o flag para indicar se a chuva é extrema ou não</h2>

In [ ]:
df_intptd['heavy'] = (df_intptd['d_prcp'] > 1.6548)
df_intptd['heavy']  = df_intptd['heavy'].astype(object).replace({False: 0, True: 1})
df_intptd[:10]

<h2 style="color:blue;">Pearson correlation coefficient</h2>

In [ ]:
df_corl = df_intptd.copy(deep=True)
for v in df_corl.columns:
  if v in ['d_stn','d_da','d_mo','d_year']:
    df_corl = df_corl.drop(v, axis = 1)

df_corl.corr()[['d_prcp']].sort_values('d_prcp')  

<p>Nenhuma variável apresentou uma correlação alta. Assim vamos aplicar um modelo não linear.</p>

<h2 style="color:blue;">Treinamento e validação</h2>

In [ ]:
df_intptd[:10]

<p>Colocando a variável alvo na primeira posição do dataframe</p>

In [ ]:
TARGET = u'd_prcp'
#TARGET = u'heavy'

cols = list(df_intptd)
cols.remove(TARGET)
cols.insert(0, TARGET)
print (cols) 

In [ ]:
for v in df_intptd.columns:
  if v in ['d_da','d_mo','d_year']:
    df_intptd = df_intptd.drop(v, axis = 1)

In [ ]:
df_intptd[:12]

In [ ]:
FEATURES = df_intptd.columns.tolist()

FLAGS = ['d_fog','d_rain_drizzle','d_snow_ice_pellets','d_hail','d_thunder','d_tornado_funnel_cloud']

for f in FLAGS:
  FLAGS.append(f+'_1')
  FLAGS.append(f+'_2')
  FLAGS.append(f+'_3')

print FLAGS 

SPARSE = ['d_stn']

for f in FLAGS:
  FEATURES.remove(f)
  
print FEATURES

FEATURES.remove(SPARSE)

FEATURES.remove(TARGET)

In [ ]:
TRAIN_SIZE = int(0.7*len(df_intptd))
df_train = df_intptd[:TRAIN_SIZE]
df_valid = df_intptd[TRAIN_SIZE:]

In [ ]:
import tensorflow as tf
import shutil

print tf.__version__

<h3 style="color:red;">Funções para ler o dataframe para inserir no tf.constant</h3>

In [ ]:
def make_input_fn(df):
  def pandas_to_tf(pdcol):
    # convert the pandas column values to float
    t = tf.constant(pdcol.astype('float32').values)
    # take the column which is of shape (N) and make it (N, 1)
    return tf.expand_dims(t, -1)
  
  def input_fn():
    # create features, columns
    features = {k: pandas_to_tf(df[k]) for k in FEATURES}
    labels = tf.constant(df[TARGET].values)
    return features, labels
  return input_fn
  
def make_feature_cols():
  print FEATURES
  input_columns = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]
  return input_columns

<h3 style="color:green;">Regressão Linear</h3>

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)
shutil.rmtree('prcp', ignore_errors=True) # start fresh each time
model = tf.contrib.learn.LinearRegressor(feature_columns=make_feature_cols(), model_dir='prcp')


In [ ]:
model.fit(input_fn=make_input_fn(df_train), steps=10);

In [ ]:
def print_rmse(model, name, input_fn):
  metrics = model.evaluate(input_fn=input_fn, steps=1)
  print u'RMSE(Root Mean Squared Error) no {} = {}'.format(name, np.sqrt(metrics['loss']))
  
print_rmse(model, u'conjunto de validação', make_input_fn(df_valid))

In [ ]:
model = tf.contrib.learn.LinearRegressor(feature_columns=make_feature_cols(), model_dir='prcp')
preds = model.predict(input_fn=make_input_fn(df_valid))
m = list(preds)

<p>Comparando as predições com o conjunto de validação.</p>

In [ ]:
df_comparar = df_valid.copy(deep=True)
df_comparar = df_comparar.reset_index(drop=True)

df_comparar['resultado'] = m

cols = list(df_comparar)
cols.insert(0, cols.pop(cols.index('resultado')))
df_comparar = df_comparar.ix[:, cols]

print df_comparar[:10]


<p><b>Root Mean Square Error (RMSE)</b> is the standard deviation of the residuals (prediction errors). Residuals are a measure of how far from the regression line data points are; RMSE is a measure of how spread out these residuals are. In other words, it tells you how concentrated the data is around the line of best fit. Root mean square error is commonly used in climatology, forecasting, and regression analysis to verify experimental results.</P>

<h3 style="color:green;"> Deep Neural Network regression </h3>

In [ ]:
shutil.rmtree('prcp', ignore_errors=True)
model = tf.contrib.learn.DNNRegressor(hidden_units=[32, 8, 2], feature_columns=make_feature_cols(), 
                                      optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.3,l1_regularization_strength=0.001),
                                      model_dir='prcp', )
model.fit(input_fn=make_input_fn(df_train), steps=500);
print_rmse(model, u'conjunto de validação', make_input_fn(df_valid))
preds = model.predict(input_fn=make_input_fn(df_valid))
m = list(preds)

In [ ]:
df_comparar = df_valid.copy(deep=True)
df_comparar = df_comparar.reset_index(drop=True)

df_comparar['resultado'] = m

cols = list(df_comparar)
cols.insert(0, cols.pop(cols.index('resultado')))
df_comparar = df_comparar.ix[:, cols]

print df_comparar[:10]

<h2 style="color:blue;">Conclusão</h2>

<p>Os resultados não foram bons, visto que são viciados, mas a proposta é rodar novamente mas desta vez aplicando validação cruzada. E refazer o holdout mas feito corretamente, ou seja, selecionar randomicamente. Aplicando o modelo regressão linear o <b>RMSE(Root Mean Squared Error) foi 22.68 </b> e aplicando redes neurais <b>(DNN Regressor) foi 24.66 </b>.</p>

<p>Os próximos passos são fazer o balanciamento, e possivelmente discretizar a chuva extrema em intervalos ao inves de prever a precipitação em valores contínuos. E selecionar as instâncias de forma randômica.</p>

